In [1]:
from keras.layers import Conv2D, Activation, BatchNormalization, SeparableConv2D,MaxPooling2D,GlobalAveragePooling2D
from keras import Model
from keras import layers

Using TensorFlow backend.


In [2]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model

In [3]:
from keras.preprocessing.image import ImageDataGenerator

In [4]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [5]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [7]:
import pickle

In [8]:
with open("faces_dataset_conv2.pickle", "rb") as f:
    data = pickle.load(f)

In [9]:
train_ds = data["train_dataset"]
train_labels = data["train_labels"]
pub_test_ds = data["public_test"]
pub_test_labels = data["public_test_labels"]
priv_test_ds = data["private_test"]
priv_test_labels = data["private_test_labels"]

In [10]:
train_ds = train_ds.reshape(train_ds.shape[0], 48, 48, 1)
pub_test_ds = pub_test_ds.reshape(pub_test_ds.shape[0],48, 48, 1)
priv_test_ds = priv_test_ds.reshape(priv_test_ds.shape[0], 48, 48, 1)

In [11]:
from keras.utils import np_utils

In [37]:
faces.shape

(35887, 48, 48, 1)

In [12]:
train_labels = np_utils.to_categorical(train_labels, 7)
pub_test_labels = np_utils.to_categorical(pub_test_labels, 7)
priv_test_labels = np_utils.to_categorical(priv_test_labels, 7)

In [15]:
train_big = np.vstack([train_ds, pub_test_ds])

In [16]:
train_label_big = np.concatenate([train_labels, pub_test_labels])

In [17]:
patience = 50

In [18]:
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
model_names ='weights_mini_xception.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [19]:
batch_size = 64
num_epochs = 10

In [20]:
from keras.regularizers import l2
from keras.layers import Input

In [60]:
model = mini_XCEPTION((48, 48, 1), 7)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
model.fit_generator(data_generator.flow(train_big, train_label_big,
                                            batch_size),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=[priv_test_ds, priv_test_labels])
model.save_weights("weights_10_epochs_xception.hdf5")

Epoch 1/10
505/504 [==============================] - 50s 99ms/step - loss: 1.7405 - acc: 0.3378 - val_loss: 1.6357 - val_acc: 0.3987

Epoch 00001: val_loss improved from inf to 1.63574, saving model to weights_mini_xception.01-0.40.hdf5
Epoch 2/10
505/504 [==============================] - 41s 82ms/step - loss: 1.5076 - acc: 0.4338 - val_loss: 1.4943 - val_acc: 0.4553

Epoch 00002: val_loss improved from 1.63574 to 1.49432, saving model to weights_mini_xception.02-0.46.hdf5
Epoch 3/10
505/504 [==============================] - 42s 83ms/step - loss: 1.3932 - acc: 0.4817 - val_loss: 1.3591 - val_acc: 0.4753

Epoch 00003: val_loss improved from 1.49432 to 1.35915, saving model to weights_mini_xception.03-0.48.hdf5
Epoch 4/10
505/504 [==============================] - 44s 87ms/step - loss: 1.3166 - acc: 0.5095 - val_loss: 1.2991 - val_acc: 0.5130

Epoch 00004: val_loss improved from 1.35915 to 1.29906, saving model to weights_mini_xception.04-0.51.hdf5
Epoch 5/10
505/504 [================

In [63]:
batch_size = 64
num_epochs = 100

In [64]:
model.fit_generator(data_generator.flow(train_big, train_label_big,
                                            batch_size),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=[priv_test_ds, priv_test_labels])
model.save_weights("weights_110_epochs_xception.hdf5")

Epoch 1/100
505/504 [==============================] - 42s 82ms/step - loss: 1.1386 - acc: 0.5742 - val_loss: 1.2242 - val_acc: 0.5561

Epoch 00001: val_loss did not improve
Epoch 2/100
505/504 [==============================] - 42s 83ms/step - loss: 1.1279 - acc: 0.5770 - val_loss: 1.3232 - val_acc: 0.5107

Epoch 00002: val_loss did not improve
Epoch 3/100
505/504 [==============================] - 45s 89ms/step - loss: 1.1185 - acc: 0.5808 - val_loss: 1.2328 - val_acc: 0.5514

Epoch 00003: val_loss did not improve
Epoch 4/100
505/504 [==============================] - 46s 90ms/step - loss: 1.1032 - acc: 0.5850 - val_loss: 1.1673 - val_acc: 0.5701

Epoch 00004: val_loss did not improve
Epoch 5/100
505/504 [==============================] - 47s 94ms/step - loss: 1.0953 - acc: 0.5937 - val_loss: 1.0878 - val_acc: 0.5885

Epoch 00005: val_loss improved from 1.16476 to 1.08784, saving model to weights_mini_xception.05-0.59.hdf5
Epoch 6/100
505/504 [==============================] - 48s 94

In [30]:
model = mini_XCEPTION((48, 48, 1), 7)
model.load_weights("weights_mini_xception.62-0.65.hdf5")
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.evaluate(priv_test_ds, priv_test_labels)

3589/3589 [==============================] - 2s 472us/step


[0.9596387829229871, 0.6531067149789926]

In [32]:
import time

In [40]:
start = time.time()
model.predict(np.array([priv_test_ds[2]]))
print(start - time.time())

-0.004477500915527344
